In [1]:
import dagshub
from dagshub.common.api.repo import RepoAPI
from io import BytesIO
from pathlib import Path
from PIL import Image
import requests
from requests.auth import HTTPBasicAuth
import tifffile

import plotly.express as px
import plotly.graph_objects as go

#auth = HTTPBasicAuth('zndr27', '')
token = '9379eb5d9f33527968c7253f77074ac56d67d9e4'
headers = {"Authorization": f"Bearer {token}"}

In [2]:
requests.request('GET', url = "https://dagshub.com/api/v1/user", headers=headers)

<Response [200]>

In [3]:
requests.request('GET', 'https://dagshub.com/api/v1/repos/OperationSavta/SavtaDepth', headers=headers)

<Response [200]>

In [4]:
gensum = requests.request('GET', 'https://dagshub.com/OperationSavta/SavtaDepth/experiments/api', headers=headers)

In [5]:
gensum.json()

{'experiments': {'470905': {'ID': '470905',
   'RepoID': 713,
   'CommitID': '3ca0495c29e0777de70a568966fcf2035cf71e77',
   'TimeUnixMs': 1679435351000,
   'Tags': {'CommitAuthorName': 'RihabFekii',
    'CommitAuthorTimestampUnixSec': 1679435351,
    'CommitCommitterName': 'RihabFekii',
    'CommitCommitterTimestampUnixSec': 1679435351,
    'CommitMessage': 'creating a bentoml service\n'},
   'Name': '',
   'MetricsByPath': {'logs/test_metrics.csv': {'LatestByName': {'a1': {'ExperimentID': '470905',
       'Name': 'a1',
       'Value': 0.6736493,
       'Timestamp': 1615292013951,
       'Step': 1,
       'ValueStr': '0.6736493',
       'Path': 'logs/test_metrics.csv',
       'IsInvalidFloat': False},
      'a2': {'ExperimentID': '470905',
       'Name': 'a2',
       'Value': 0.9112526,
       'Timestamp': 1615292013951,
       'Step': 1,
       'ValueStr': '0.9112526',
       'Path': 'logs/test_metrics.csv',
       'IsInvalidFloat': False},
      'a3': {'ExperimentID': '470905',
     

In [6]:
italy_get_experiments = requests.request('GET', 'https://dagshub.com/api/v1/repos/Omdena/TriesteItalyChapter_MappingSeagrassMeadows/experiments', headers=headers)
print(italy_get_experiments)

<Response [200]>


In [7]:
italy_get_experiments = requests.request('GET', 'https://dagshub.com/Omdena/TriesteItalyChapter_MappingSeagrassMeadows.mlflow/api/2.0/mlflow/experiments/list', headers=headers)
print(italy_get_experiments)

<Response [401]>


In [8]:
italy_get_info = requests.request('GET', 'https://dagshub.com/api/v1/repos/Omdena/TriesteItalyChapter_MappingSeagrassMeadows', headers=headers)
print(italy_get_info)

<Response [200]>


In [9]:
repo = RepoAPI('Omdena/TriesteItalyChapter_MappingSeagrassMeadows')

In [10]:
repo_info = repo.get_repo_info()

In [11]:
path_list = {}
for path in repo.list_path(''):
    path_list.setdefault(path.versioning, []).append(path)

In [12]:
bucket_list = path_list.get('bucket', [])
dvc_list = path_list.get('dvc', [])
git_list = path_list.get('git', [])

In [13]:
storage_list = repo.get_connected_storages()

In [14]:
storage = next(iter(storage_list))

In [15]:
storage

StorageAPIEntry(name='TriesteItalyChapter_MappingSeagrassMeadows', protocol='s3', list_path='https://dagshub.com/api/v1/repos/Omdena/TriesteItalyChapter_MappingSeagrassMeadows/storage/content/s3/TriesteItalyChapter_MappingSeagrassMeadows')

In [16]:
repo.list_storage_path('s3/TriesteItalyChapter_MappingSeagrassMeadows', True)

[]

In [17]:
dvc_dir = next(iter(dvc_list))

In [18]:
images = repo.list_path('data/Seagrass_ROI_Summer/images')
img = images[0]

In [19]:
repo_api_path = Path('https:/dagshub.com/api/v1/repos')
repo_id = Path('./Omdena/TriesteItalyChapter_MappingSeagrassMeadows')
xtra_path = Path('./content/main')
img_path = Path(img.content_url)
img_path_rel = img_path.relative_to(repo_api_path / repo_id / xtra_path)

In [20]:
image_bytes = repo.get_file(str(img_path_rel))

In [21]:
with open(img_path.name, 'wb') as f:
    f.write(image_bytes)

In [22]:
image_io = BytesIO(image_bytes)
image_io.seek(0)

0

In [23]:
image_path = Path('data/Seagrass_ROI_Summer/images/s_Greece_04_image_0.tif')
mask_path = Path('data/Seagrass_ROI_Summer/masks/s_Greece_04_mask_0.tif')
image_bytes = repo.get_file(str(image_path))
mask_bytes = repo.get_file(str(mask_path))
with open(image_path.name, 'wb') as f:
    f.write(image_bytes)
with open(mask_path.name, 'wb') as f:
    f.write(mask_bytes)

In [24]:
def read_image_from_bytes(image_path):
    try:
        return Image.open(image_path)
    except:
        pass
    try:
        return cv2.read(image_path)
    except:
        pass
    try:
        return tifffile.imread(image_path)
    except:
        pass
    return None

In [25]:
import skimage
image = skimage.io.imread(image_path.name)
mask = skimage.io.imread(mask_path.name)

In [26]:
image

array([[[ 902,  899,  843, ...,  683,  727,  668],
        [ 902,  836,  876, ...,  683,  726,  681],
        [ 902,  836,  876, ...,  683,  726,  681],
        ...,
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0]],

       [[ 902,  883,  868, ...,  683,  727,  664],
        [ 902,  846,  896, ...,  683,  729,  679],
        [ 902,  846,  896, ...,  683,  729,  679],
        ...,
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0]],

       [[ 902,  835,  874, ...,  683,  727,  664],
        [ 902,  893,  943, ...,  683,  729,  679],
        [ 902,  893,  943, ...,  683,  729,  679],
        ...,
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0]],

       ...,

       [[ 993, 1066, 117